### Linear regression

In [1]:
import math

def golden (x):
    return 2 + 3*x

def fit (a=0.0, b=0.0, epsilon, n=10000, attempts=0, lrate=0.01):
    prev_error = 0.0
    while(True):
        if error(a) < epsilon: return a
        if attempts >= n: return a
        attempts += 1
        
        crnt_error = error(a,b)
        diff=crnt_error-prev_error
        a-=
    
def error (a, b, n=1000):
    error = 0.0;
    for i in range(0,n):
        x = float(i)/n
        error += (golden(x) - (a + b*x))**2
    error /= n 
    return math.sqrt(error)

fit(1, 0.01, prev_error=100)
        

SyntaxError: invalid syntax (<ipython-input-1-e093bfa55c6e>, line 15)

### Jacobian Method
Basic steps in a Jacobi eigenvalue procedure:
1. Choose an index pair $(p,q)$ that satisfies $1 \leq p \lt q \leq n$
2. Compute a cosine-sine pair such that 

$$
\begin{bmatrix}
b_{pp} & b_{pq}\\
b_{qp} & b_{qq}
\end{bmatrix}
=
\begin{bmatrix}
c & s\\
-s & c
\end{bmatrix}^T
\begin{bmatrix}
a_{pp} & a_{pq}\\
a_{qp} & a_{qq}
\end{bmatrix}
\begin{bmatrix}
c & s\\
-s & c
\end{bmatrix}
$$
is diagonal.
3. Overwrite $A$ with $B=J^TAJ$ where $J=J(p,q,\theta)$. Observe that $B$ agrees with $A$ except for rows and columns $p$ and $q$
#### Error function
$off(B)^2 = ||B||^2_F - \sum_{i=1}^{n}{b^2_{ii}} = off(A)^2 - 2a^2_{pq}$

#### 2-by-2 Symmetric Schur Decomposition
To say that we diagonalize in (step 2) is to say that 
$0=b{pq}=a{pq}(c^2 - s^2) + (a_{pp} - a_{qq})cs$. 
If $a_{pq} = 0$, then we just set $(c,s)=(1,0)$. Otherwise define $t=\frac{a_{qq}-a_{pp}}{2a_{pq}}$ and $t=\frac{s}{c}$, $t=tan(\theta)$ solves the quadratic $t^2 + 2rt - 1 = 0$.
$$t = -r \pm \sqrt{1+r^2}$$
Whereupon c and s can be resolved from the formulae
$$c=1/\sqrt{1+t^2}$$ $$s=tc$$
Choosing t to be the smaller of the two roots ensures that $|\theta|\leq \pi/4$ and has the effect of minimizing the difference between B and A.
![title](figures/symschurdec.png)

In [1]:
import numpy as np
import math

def sym_schur2(A, p, q):
    if (A[p,q] != 0):
        t = ((A[q,q] - A[p,p]) / (2*A[p,q]))
        t = 1/(t+math.sqrt(1+t**2)) if (t>=0) else -1/(-t + math.sqrt(1+t**2))
        c = 1/math.sqrt(1+t**2)
        s = t*c
    else:
        c = 1
        s = 0
    return c, s

# Test
A = np.array([[1,2,3],[2,4,5],[3,5,6]])
p = 1; q = 1;
sym_schur2(A, p, q)

(0.7071067811865475, 0.70710678118654746)

#### The classical Jacobi Algorithm
![title](figures/classjac.png)

In [2]:
# Cost function
def off(A):
    sum_ = 0
    for i in range(0, len(A[:,1])): # Rows
        for j in range(0, len(A[1,:])): # Columns
           sum_ += A[i,j]**2 if (i!=j) else 0
    return math.sqrt(sum_)

#p: row, q: column
def cyclicPQ(size, A=0, p=0, q=0):
    # Increment q if possible
    if (q < p-1):
        q += 1
    elif (p<size-1):
        p += 1
        q = 0
    else:
        p=0;q=1;
    return (p, q)

def optimalPQ(size, A, p=0, q=0):
    # Find biggest off-diagonal element of A
    p=0; q=0; resP=0; resQ=1; maxElement=0; 
    while q<(size-2):
        p, q = cyclicPQ(size, p=p, q=q) # Iterate through off-diagonal elements
        if (A[p,q] > maxElement):
            maxElement = A[p,q]
            resP, resQ = p, q
    #print("Chose (P,Q) = ({},{})".format(resP, resQ))
    return resP, resQ
    
# Classical Jacobi
def jac(A, tol=0.01, choosePQ=cyclicPQ):
    size = A.shape[0]
    V = np.identity(size)
    epsilon = tol*np.linalg.norm(A) # Frobernius norm, i.e. sum of singular values (root of square sum)
    print("Epsilon: {}".format(epsilon))
    p=0; q=0; iterations=0; sweeps = 0
    print("i\toff(A)\n{:-<32}".format(''))
    print("{}\t{}".format(sweeps, off(A)))
    while off(A) > epsilon:
        p, q = choosePQ(size, A, p, q)
        c, s  = sym_schur2(A,p,q)
        # Calculate rotation matrix
        R = np.identity(size)
        R[p,p]=c;  R[p,q]=s;
        R[q,p]=-s; R[q,q]=c;
        # Calculate next A
        RT = np.transpose(R)
        A = np.matmul(RT, A) # R^T * A
        A = np.matmul(A, R) # A*R
        # Refine V - Eigenvector matrix
        V = np.matmul(V,R)
        iterations += 1
        sweeps = int(iterations/size)
        if (iterations%size==0):
            print("{}\t{:.3f}".format(sweeps, off(A)))
    print("{}\t{:.3f}".format(sweeps, off(A)))
    return (A, V)
        
# Test
A = np.array([[1,1,1,1], 
              [1,2,3,4],
              [1,3,6,10],
              [1,4,10,20]])
E, V = jac(A, choosePQ=optimalPQ)
print(E)
print(V)
p, q = optimalPQ(A.shape[0], A)
        

Epsilon: 0.264007575649
i	off(A)
--------------------------------
0	16.0
1	1.624
1	0.249
[[  4.05385980e-01   2.02551857e-04  -1.33561958e-01  -5.51950087e-02]
 [  2.02551857e-04   2.20015976e+00  -8.31318740e-02   6.53990750e-16]
 [ -1.33561958e-01  -8.31318740e-02   8.99923142e-02  -5.72262468e-02]
 [ -5.51950087e-02  -1.38777878e-17  -5.72262468e-02   2.63044620e+01]]
[[ 0.84685273  0.5282037   0.          0.06197832]
 [-0.39903263  0.61641666 -0.64902773  0.19891325]
 [-0.29660868  0.42168262  0.72353568  0.45902416]
 [ 0.18877689 -0.40399892 -0.23507261  0.86364867]]
